In [1]:

import polars as pl 
import os


In [14]:
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
file_name = os.path.join("..", "data", "source", "defilama_0813.xlsx")
# Try reading the Excel file with modified parameters
chain_info = [
    {"chain_id": 1, "chain_name": "ethereum"},
    {"chain_id": 56, "chain_name": "binance_smart_chain"},
    {"chain_id": 137, "chain_name": "polygon"},
    {"chain_id": 8453, "chain_name": "base"},
    {"chain_id": 59144, "chain_name": "linea"},
    {"chain_id": 42161, "chain_name": "arbitrum"},
    {"chain_id": 534352, "chain_name": "scroll"},
    {"chain_id": 10, "chain_name": "optimism"},
    {"chain_id": 204, "chain_name": "opbnb"},
    {"chain_id": 167000, "chain_name": "taiko"},
    {"chain_id": 34443, "chain_name": "mode"},
    {"chain_id": 43114, "chain_name": "avalanche"},
    {"chain_id": 250, "chain_name": "fantom"},
    {"chain_id": 81457, "chain_name": "blast"},
    {"chain_id": 200901, "chain_name": "bitlayer"},
    {"chain_id": 196, "chain_name": "xlayer"},
    {"chain_id": 5000, "chain_name": "mantle"}
]
df2 = pl.DataFrame(chain_info)
print(df2)
# protocol_id	protocol_name	tvl	version_name	chain_id	contract_address	method_id	tx_cnt	tx_hash1	tx_hash2	description	rk

SchemaDict = {
    "protocol_id": pl.Int64,
    "protocol_name": pl.Utf8,
    "tvl": pl.Float64,
    "version_name": pl.Utf8,
    "chain_id": pl.Int64,
    "contract_address": pl.Utf8,
    "method_id": pl.Utf8,
    "tx_cnt": pl.Int64,
}
df1=pl.read_excel(
        source=file_name,
        schema_overrides=SchemaDict,
        infer_schema_length=None,
    ) 
ctx = pl.SQLContext()
ctx.register("df1", df1)
ctx.register("df2", df2)
sql = """
    select protocol_name,chain_id,contract_address,method_id,tx_cnt 
    from df1
    where protocol_name in 
        ('gmx', 'curve', 'layerbank', 'pendle')
    order by protocol_name, tvl desc, chain_id, tx_cnt desc
"""

sql = """
    with required_chains as (
        select df1.chain_id
        from df1
        group by df1.chain_id
    ),
    supported_chains as (
        select df2.chain_id
        from df2
    ),
    unsupported_chains as (
        select required_chains.chain_id
        from required_chains
        where required_chains.chain_id not in (select supported_chains.chain_id from supported_chains)
    )
    select * from unsupported_chains
"""
res = ctx.execute(sql).collect()
res

shape: (17, 2)
┌──────────┬─────────────────────┐
│ chain_id ┆ chain_name          │
│ ---      ┆ ---                 │
│ i64      ┆ str                 │
╞══════════╪═════════════════════╡
│ 1        ┆ ethereum            │
│ 56       ┆ binance_smart_chain │
│ 137      ┆ polygon             │
│ 8453     ┆ base                │
│ 59144    ┆ linea               │
│ …        ┆ …                   │
│ 250      ┆ fantom              │
│ 81457    ┆ blast               │
│ 200901   ┆ bitlayer            │
│ 196      ┆ xlayer              │
│ 5000     ┆ mantle              │
└──────────┴─────────────────────┘


chain_id
i64
1116
100
8217
223
42170
…
25
1030
14


In [36]:
import json

# Read the JSON file
file = os.path.join("..", "data", "common", "chain_info.json")
with open(file, 'r') as f:
    data = json.load(f)

# Access the nested data
chains = data['props']['pageProps']['chains']

# Convert to a Polars DataFrame if needed
df = pl.DataFrame(chains)
ctx = pl.SQLContext()
ctx.register("df", df)
sql = """
    select chainId,name as chain_name, shortName
    from df
"""
res = ctx.execute(sql).collect()

res.write_csv(os.path.join("..", "data", "common", "chain_info.csv"))
res.shape

(1729, 3)